# Coding for Economists - Session 9 & 10

***

## 1. Setup Environment

In [ ]:
%pip install econml

In [ ]:
%pip install --upgrade jupyter ipywidgets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Turn on copy on write
pd.options.mode.copy_on_write = True

## 2. K-Means Clustering

### 2.1 Read Data

In [ ]:
# Load the publicly available macroeconomic dataset
import statsmodels.api as sm
data = sm.datasets.macrodata.load_pandas().data

# Select a subset of economic indicators for clustering
# Here we use real GDP, inflation, and unemployment
df = data[['realgdp', 'infl', 'unemp']]
df.head()

### 2.2 Preprocessing

In [ ]:
# Scale the data (important for clustering)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

### 2.3 Determine the Optimal Number of Clusters

In [ ]:
# Determine the optimal number of clusters using the elbow method
from sklearn.cluster import KMeans
inertia = []
K_range = range(1, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1234)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve (within cluster sum of squares)
plt.figure(figsize=(8, 4))
plt.plot(K_range, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
# Determine optimal k using silhouette scores (requires k>=2)
from sklearn.metrics import silhouette_score
silhouette_scores = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(scaled_data)
    score = silhouette_score(scaled_data, labels)
    silhouette_scores.append(score)

# Plot the silhouette scores
plt.figure(figsize=(8, 4))
plt.plot(range(2, 11), silhouette_scores, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for Different k')
plt.show()

In [ ]:
# Based on the elbow and silhouette methods, choose an optimal k
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(scaled_data)

### 2.4 Visualize the Clusters

In [ ]:
# Add cluster labels to the dataframe
df['cluster'] = clusters

# Use PCA to reduce dimensions to 2 for visualization purposes
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=1234)
pca_result = pca.fit_transform(scaled_data)
df['pca1'] = pca_result[:, 0]
df['pca2'] = pca_result[:, 1]
df.head()

In [ ]:
# Plot the clusters in the PCA-reduced space
plt.figure(figsize=(8, 6))
for cluster in range(optimal_k):
    subset = df[df['cluster'] == cluster]
    plt.scatter(subset['pca1'], subset['pca2'], label=f'Cluster {cluster}')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title(f'K-Means Clustering (k={optimal_k})')
plt.legend()
plt.show()

## 3. Policy Evaluation: Causal Forest

### 3.1 Read Data

In [ ]:
df = pd.read_csv('lalonde_data.csv', index_col=0)
print(df.shape)
df.head()

- __treat__: 1 participated in job training program, 0 otherwise
- __age__: measured in years;
- __educ__: measured in years;
- __black__: indicating race (1 if black, 0 otherwise);
- __hispan__: indicating race (1 if Hispanic, 0 otherwise);
- __married__: indicating marital status (1 if married, 0 otherwise);
- __nodegree__: indicating high school diploma (1 if no degree, 0 otherwise);
- __re74__: real earnings in 1974;
- __re75__: real earnings in 1975;
- __re78__: real earnings in 1978.

In [ ]:
df.describe()

### 3.2 Data Cleaning

In [ ]:
print("Missing values per column:\n", df.isnull().sum())

### 3.3 Preprocessing

In [ ]:
df.dtypes

### 3.4 Prepare Train/Test Sets

In [ ]:
# Define outcome (Y), treatment (T), and covariates (X)
Y = df['re78'].values       # outcome variable: earnings in 1978
T = df['treat'].values      # treatment indicator
# Use remaining columns as covariates
X = df.drop(columns=['treat', 're78']).values

# Split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(
    X, Y, T, test_size=0.2, random_state=1234
)

### 3.5 Implement Causal Forest and Tune Parameters

#### Define Hyperparameter Grid for Tuning
- __n_estimators__: The number of trees in the forest.
- __max_depth__: The maximum depth of the tree.
- __min_samples_split__: The minimum number of samples required to split an internal node.
- __max_features__: The number of features to consider when looking for the best split.

In [ ]:
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

best_model = None
best_params = None
best_cate_std = np.inf

#### Grid Search

In [ ]:
from sklearn.model_selection import ParameterGrid
from econml.dml import CausalForestDML
# For causal inference, one approach is to choose the hyperparameters that provide stable (low-variance) CATE estimates.
# Here, as a demo, we use the standard deviation of the estimated conditional Treatment Effects (TE)
# on the test set as a proxy (lower variance can indicate more stable estimates).

for params in ParameterGrid(param_grid):
    model = CausalForestDML(
        model_y='forest',
        model_t='forest',
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        max_features=params['max_features'],
        random_state=1234,
        verbose=0
    )
    model.fit(Y_train, T_train, X=X_train)
    # Estimate the CATE on the test set
    cate_test = model.effect(X_test)
    cate_std = np.std(cate_test)
    print(f"Params: {params}, CATE std: {cate_std:.2f}")
    
    if cate_std < best_cate_std:
        best_cate_std = cate_std
        best_params = params
        best_model = model

print("\nBest Hyperparameters:", best_params)

In [ ]:
from econml.cate_interpreter import SingleTreeCateInterpreter
plt.figure(figsize=(8, 6))
intrp = SingleTreeCateInterpreter(max_depth=2).interpret(best_model, X_train)
intrp.plot(feature_names=df.columns.values[1:-1], fontsize=12)

### 3.6 Validate the Model

In [ ]:
# Estimate the Average Treatment Effect (ATE) and its confidence interval on the test set.
ate_point = best_model.ate(X_test)
ate_lb, ate_ub = best_model.ate_interval(X_test, alpha=0.05)
print(f"\nEstimated ATE: {ate_point:.2f}")
print(f"95% Confidence Interval for ATE: ({ate_lb:.2f}, {ate_ub:.2f})")

# Also, compute CTE for each test observation
cte_pred = best_model.effect(X_test)

### 3.7 Visualization

In [ ]:
# Plot 1: Histogram of Estimated CTE
plt.figure(figsize=(8, 5))
plt.hist(cate_pred, bins=30, edgecolor='k', alpha=0.7)
plt.title("Distribution of Estimated Conditional Treatment Effects (CATE)")
plt.xlabel("Estimated Treatment Effect")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
# Plot 2: Scatter Plot of CTE vs. a Covariate (e.g., Age)
# Assuming the first column in X corresponds to age
age_test = X_test[:, 0]
plt.figure(figsize=(8, 5))
plt.scatter(age_test, cate_pred, alpha=0.6, edgecolor='k')
plt.title("Estimated Treatment Effect vs. Age")
plt.xlabel("Age")
plt.ylabel("Estimated Treatment Effect")
plt.grid(True)
plt.show()

In [ ]:
# Plot 3: Visualize the ATE with its Confidence Interval
plt.figure(figsize=(6, 4))
plt.errorbar(1, ate_point, yerr=[[ate_point - ate_lb], [ate_ub - ate_point]], fmt='o', color='red', capsize=5)
plt.xlim(0.5, 1.5)
plt.xticks([1], ['ATE'])
plt.title("Average Treatment Effect with 95% Confidence Interval")
plt.ylabel("Treatment Effect")
plt.grid(True)
plt.show()

## 4. Debugging

In [ ]:
# Reset the workspace
%reset -f

### 4.1 Trace Errors

In [ ]:
# Define two numbers
a = 10
b = 5

# Print the sum
print("Sum:", a + b)
b = 0

# This line will cause a ZeroDivisionError intentionally
error_line = a / 0  # Intentional error: division by zero

# This line will not execute because of the error above
print("This line will not be printed.")

### 4.2 Jupyter Debugger
https://blog.jupyter.org/a-visual-debugger-for-jupyter-914e61716559

In [ ]:
pets = ['Cat', 'Dog', 'Cat', 'Dog', 'Cat']
pet_names = ['Whiskers', 'Buddy', 'Mittens', 'Rex', 'Shadow']
owner_emails = [
    'alice@example.com',
    'bob@example.com',
    'carol@example.com',
    'dave@example.com',
    'eve@example.com'
]
visited_last_month = [1, 0, 1, 1, 0]  # 1 = Yes, 0 = No

In [ ]:
reminder = {}
for pet, name, email, visit in zip(pets, pet_names, owner_emails, visited_last_month):
    if visit == 0:
        reminder[name] = {'Type': pet, 'Email': email}

### 4.3 try-except

In [ ]:
try:
    # Attempt to divide by zero, which will raise an exception.
    result = 10 / 0
    print("Result:", result)
except ZeroDivisionError as e:
    # This block executes if a ZeroDivisionError is raised.
    print("Caught an error:", e)
else:
    # This block executes if no exception is raised.
    print("Division was successful!")
finally:
    # This block always executes, regardless of whether an exception was raised.
    print("Execution of the try-except block is complete.")

### 4.4 Unit Testing

In [ ]:
import unittest

# Define the function to be tested
def add_numbers(a, b):
    """Return the sum of two numbers."""
    return a + b

# Define a TestCase class with test methods
class TestAddNumbers(unittest.TestCase):
    
    def test_add_positive(self):
        # Test with positive numbers
        self.assertEqual(add_numbers(2, 3), 5)
    
    def test_add_negative(self):
        # Test with negative numbers
        self.assertEqual(add_numbers(-1, -1), -2)
    
    def test_add_mixed(self):
        # Test with a mix of positive and negative numbers
        self.assertEqual(add_numbers(-1, 5), 4)

# Run the tests
unittest.main(argv=[''], exit=False)